# SEM Campaign Generator

As you might know before getting into Data Science I was a Marketing Manager.
An important part of our SEM campaigns was actually to create them, based on a good keyword and ad copy research and brainstorming. 

I got this project idea from a datacamp tutorial, I decided to apply it to a company I use quite often: Deliveroo. 

I am using BeautifulSoup to extract data from their webpage.
The idea here it's not creating the perfect ads because that would have required me to do a proper keyword search, but rather focussing on the technical side of it. 

**How can I use python to save time on such marketing tasks?**

Let's go!

![](restaurants.jpg)

**Imports**

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
page = 'https://deliveroo.de/en/'
resp = requests.get(page)
soup = BeautifulSoup(resp.text, 'lxml')




**Select the piece of HTML code that holds the information we need**

In [2]:
allcuisine = soup.find_all("div", class_="e2e5cde65adf4422")
allcuisine

[<div class="e2e5cde65adf4422"><h2 class="c1a36a7fabe35b74">Need some more options? </h2><ul class="e9048103aff13064"><li><a href="/en/pizza-takeaway">pizza</a></li><li><a href="/en/lebanese-takeaway">lebanese</a></li><li><a href="/en/american-takeaway">american</a></li><li><a href="/en/indian-takeaway">indian</a></li><li><a href="/en/vegetarian-takeaway">vegetarian</a></li><li><a href="/en/chinese-takeaway">chinese</a></li><li><a href="/en/dessert-takeaway">dessert</a></li><li><a href="/en/greek-takeaway">greek</a></li><li><a href="/en/sushi-takeaway">sushi</a></li><li><a href="/en/thai-takeaway">thai</a></li><li><a href="/en/italian-takeaway">italian</a></li><li><a href="/en/mexican-takeaway">mexican</a></li><li><a href="/en/japanese-takeaway">japanese</a></li></ul></div>]

**Extracting the urls and cuisine type**

In [3]:
url=[]
cuisine = []
for x in allcuisine:
    for a in x.find_all("a"):
        u = 'https://deliveroo.de'+ a['href'] +'/berlin'
        url.append(u)
        cuisine.append(a.get_text())

In [4]:
url

['https://deliveroo.de/en/pizza-takeaway/berlin',
 'https://deliveroo.de/en/lebanese-takeaway/berlin',
 'https://deliveroo.de/en/american-takeaway/berlin',
 'https://deliveroo.de/en/indian-takeaway/berlin',
 'https://deliveroo.de/en/vegetarian-takeaway/berlin',
 'https://deliveroo.de/en/chinese-takeaway/berlin',
 'https://deliveroo.de/en/dessert-takeaway/berlin',
 'https://deliveroo.de/en/greek-takeaway/berlin',
 'https://deliveroo.de/en/sushi-takeaway/berlin',
 'https://deliveroo.de/en/thai-takeaway/berlin',
 'https://deliveroo.de/en/italian-takeaway/berlin',
 'https://deliveroo.de/en/mexican-takeaway/berlin',
 'https://deliveroo.de/en/japanese-takeaway/berlin']

In [5]:
cuisine

['pizza',
 'lebanese',
 'american',
 'indian',
 'vegetarian',
 'chinese',
 'dessert',
 'greek',
 'sushi',
 'thai',
 'italian',
 'mexican',
 'japanese']

**Create a DataFrame to store the information**

In [6]:
df = pd.DataFrame({'name': cuisine,'url': url})
df.head()

,name,url
0,pizza,https://deliveroo.de/en/pizza-takeaway/berlin
1,lebanese,https://deliveroo.de/en/lebanese-takeaway/berlin
2,american,https://deliveroo.de/en/american-takeaway/berlin
3,indian,https://deliveroo.de/en/indian-takeaway/berlin
4,vegetarian,https://deliveroo.de/en/vegetarian-takeaway/be...


**Create a function to generate the final keyword dataframe**

In [7]:
def generate_keywords(topics, keywords, match_types=['Exact', 'Phrase', 'Broad'], campaign='SEM_cuisine'):
    col_names = ['Campaign', 'Ad Group', 'Keyword', 'Criterion Type']
    campaign_keywords = []

    for topic in topics:
        for word in keywords:
            for match in match_types:
                if match == 'Broad':
                    keyword = '+' + ' +'.join([topic.lower().replace(' ', ' +'), word.replace(' ', ' +')])
                else:
                    keyword = topic.lower() + ' ' + word
                row = [campaign,  # campaign name
                       topic,  # ad group name
                       keyword, # cuisine
                       match]  # keyword match type
                campaign_keywords.append(row)
          
    for topic in topics:
        for word in keywords:
            for match in match_types:
                if match == 'Broad':
                    keyword = '+' + ' +'.join([word.replace(' ', ' +'), topic.lower().replace(' ', ' +')])
                else:
                    keyword = word + ' ' + topic.lower()
                row = [campaign,  # campaign name
                       topic,  # ad group name
                       keyword, # cuisine
                       match]  # keyword match type
                campaign_keywords.append(row)

    return pd.DataFrame.from_records(campaign_keywords, columns=col_names)

**Apply the custom function**

In [8]:
topics = df['name']
keywords = ['eat', 'take away', 'delicious', 'food', 'food delivery', 'deliveroo', 'berlin','order','for dinner','for lunch','restaurant']
df_keywords = generate_keywords(topics, keywords)
df_keywords.head()

,Campaign,Ad Group,Keyword,Criterion Type
0,SEM_cuisine,pizza,pizza eat,Exact
1,SEM_cuisine,pizza,pizza eat,Phrase
2,SEM_cuisine,pizza,+pizza +eat,Broad
3,SEM_cuisine,pizza,pizza take away,Exact
4,SEM_cuisine,pizza,pizza take away,Phrase


**Add the campaign field to the df dataframe**

In [10]:
df['Campaign'] = 'SEM_cuisine'
df.head()


,name,url,Campaign
0,pizza,https://deliveroo.de/en/pizza-takeaway/berlin,SEM_cuisine
1,lebanese,https://deliveroo.de/en/lebanese-takeaway/berlin,SEM_cuisine
2,american,https://deliveroo.de/en/american-takeaway/berlin,SEM_cuisine
3,indian,https://deliveroo.de/en/indian-takeaway/berlin,SEM_cuisine
4,vegetarian,https://deliveroo.de/en/vegetarian-takeaway/be...,SEM_cuisine


**Duplicate rows 3 times for each future descriptions**

In [11]:
totalgr = len(df)
df_ads = df.rename(columns={'name': 'Ad Group', 'url': 'Final URL'})#
df_ads[['Campaign', 'Ad Group', 'Final URL']]
df_ads= df_ads.iloc[[x  for x in range(totalgr) for i in range(3)], :] 
df_ads.head()

,Ad Group,Final URL,Campaign
0,pizza,https://deliveroo.de/en/pizza-takeaway/berlin,SEM_cuisine
0,pizza,https://deliveroo.de/en/pizza-takeaway/berlin,SEM_cuisine
0,pizza,https://deliveroo.de/en/pizza-takeaway/berlin,SEM_cuisine
1,lebanese,https://deliveroo.de/en/lebanese-takeaway/berlin,SEM_cuisine
1,lebanese,https://deliveroo.de/en/lebanese-takeaway/berlin,SEM_cuisine


**Apend the descriptions to each row**

In [12]:
Description = [
    'Order Food Online, fast reliable restaurants available at your door step',
    'Best restaurants, in few click and get your food at your choosen location',
    'Fancy having a different meal today? Choose among our partnership restaurants'   
]
Description = [x for i in range(totalgr) for x in Description ]
df_ads['Description'] = Description
df_ads.head()

,Ad Group,Final URL,Campaign,Description
0,pizza,https://deliveroo.de/en/pizza-takeaway/berlin,SEM_cuisine,"Order Food Online, fast reliable restaurants a..."
0,pizza,https://deliveroo.de/en/pizza-takeaway/berlin,SEM_cuisine,"Best restaurants, in few click and get your fo..."
0,pizza,https://deliveroo.de/en/pizza-takeaway/berlin,SEM_cuisine,Fancy having a different meal today? Choose am...
1,lebanese,https://deliveroo.de/en/lebanese-takeaway/berlin,SEM_cuisine,"Order Food Online, fast reliable restaurants a..."
1,lebanese,https://deliveroo.de/en/lebanese-takeaway/berlin,SEM_cuisine,"Best restaurants, in few click and get your fo..."


**Add and create an headline variable _ Print final dataframe**

In [15]:
df_ads['Headline'] = df_ads['Ad Group'] + ' Food Delivery Service'
df_ads['Headline']= df_ads['Headline'].str.title()
df_ads.head(12)

,Ad Group,Final URL,Campaign,Description,Headline 1,Headline
0,pizza,https://deliveroo.de/en/pizza-takeaway/berlin,SEM_cuisine,"Order Food Online, fast reliable restaurants a...",Pizza Food Delivery Service,Pizza Food Delivery Service
0,pizza,https://deliveroo.de/en/pizza-takeaway/berlin,SEM_cuisine,"Best restaurants, in few click and get your fo...",Pizza Food Delivery Service,Pizza Food Delivery Service
0,pizza,https://deliveroo.de/en/pizza-takeaway/berlin,SEM_cuisine,Fancy having a different meal today? Choose am...,Pizza Food Delivery Service,Pizza Food Delivery Service
1,lebanese,https://deliveroo.de/en/lebanese-takeaway/berlin,SEM_cuisine,"Order Food Online, fast reliable restaurants a...",Lebanese Food Delivery Service,Lebanese Food Delivery Service
1,lebanese,https://deliveroo.de/en/lebanese-takeaway/berlin,SEM_cuisine,"Best restaurants, in few click and get your fo...",Lebanese Food Delivery Service,Lebanese Food Delivery Service
1,lebanese,https://deliveroo.de/en/lebanese-takeaway/berlin,SEM_cuisine,Fancy having a different meal today? Choose am...,Lebanese Food Delivery Service,Lebanese Food Delivery Service
2,american,https://deliveroo.de/en/american-takeaway/berlin,SEM_cuisine,"Order Food Online, fast reliable restaurants a...",American Food Delivery Service,American Food Delivery Service
2,american,https://deliveroo.de/en/american-takeaway/berlin,SEM_cuisine,"Best restaurants, in few click and get your fo...",American Food Delivery Service,American Food Delivery Service
2,american,https://deliveroo.de/en/american-takeaway/berlin,SEM_cuisine,Fancy having a different meal today? Choose am...,American Food Delivery Service,American Food Delivery Service
3,indian,https://deliveroo.de/en/indian-takeaway/berlin,SEM_cuisine,"Order Food Online, fast reliable restaurants a...",Indian Food Delivery Service,Indian Food Delivery Service


In [16]:
df_keywords.head()

,Campaign,Ad Group,Keyword,Criterion Type
0,SEM_cuisine,pizza,pizza eat,Exact
1,SEM_cuisine,pizza,pizza eat,Phrase
2,SEM_cuisine,pizza,+pizza +eat,Broad
3,SEM_cuisine,pizza,pizza take away,Exact
4,SEM_cuisine,pizza,pizza take away,Phrase
